In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").config("spark.mongodb.input.uri","mongodb://localhost:27017/AQIdata.Realtime").config("spark.mongodb.output.uri","mongodb://locahost:27017/AQIdata.Realtime").config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").getOrCreate()
#df1 = spark1.read.format("mongo").option("uri","mongodb://localhost:27017/table.accounts").load()
#df1.show()

In [2]:
import requests
import pandas as pd
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc=SparkContext.getOrCreate();
sql_sc=SQLContext(sc)

In [3]:
response =  requests.get('https://api.data.gov.in/resource/3b01bcb8-0b14-4abf-b6f2-c1bfd384ba69?api-key=579b464db66ec23bdd000001bbf5c9af061f43ba484ad03d20dcd385&format=json&offset=0&limit=1000&filters[state]=Maharashtra')
resultJson=response.json()
json_rdd = sc.parallelize([resultJson['records']])
df = spark.read.json(json_rdd)

In [4]:
df.show(2)

+----------+-------+---+-------------------+-------------+------------+-------------+-------------+--------------+-----------+--------------------+
|      city|country| id|        last_update|pollutant_avg|pollutant_id|pollutant_max|pollutant_min|pollutant_unit|      state|             station|
+----------+-------+---+-------------------+-------------+------------+-------------+-------------+--------------+-----------+--------------------+
|Aurangabad|  India|902|14-01-2021 04:00:00|           59|       PM2.5|           86|           37|            NA|Maharashtra|More Chowk Waluj,...|
|Aurangabad|  India|903|14-01-2021 04:00:00|           82|        PM10|          103|           62|            NA|Maharashtra|More Chowk Waluj,...|
+----------+-------+---+-------------------+-------------+------------+-------------+-------------+--------------+-----------+--------------------+
only showing top 2 rows



In [5]:
from pyspark.sql.functions import to_timestamp

In [6]:
df=df.select(df.city,
             df.country,
            df.id.cast("integer"),
            to_timestamp(df.last_update, 'dd-MM-yyyy HH:mm:ss').alias('lst_update'),
            df.pollutant_avg.cast("double"),
            df.pollutant_id, 
            df.pollutant_max.cast("double"), 
            df.pollutant_min.cast("double"),
            df.state,
            df.station
            )

In [7]:
from pyspark.sql.functions import isnan, when, count, col
df.where(col('pollutant_min').isNull()).count()

18

In [8]:
df=df.na.drop()

In [9]:
df.where(col('pollutant_id').isNull()).count()

0

In [10]:
df.show(50)

+----------+-------+---+-------------------+-------------+------------+-------------+-------------+-----------+--------------------+
|      city|country| id|         lst_update|pollutant_avg|pollutant_id|pollutant_max|pollutant_min|      state|             station|
+----------+-------+---+-------------------+-------------+------------+-------------+-------------+-----------+--------------------+
|Aurangabad|  India|902|2021-01-14 04:00:00|         59.0|       PM2.5|         86.0|         37.0|Maharashtra|More Chowk Waluj,...|
|Aurangabad|  India|903|2021-01-14 04:00:00|         82.0|        PM10|        103.0|         62.0|Maharashtra|More Chowk Waluj,...|
|Aurangabad|  India|904|2021-01-14 04:00:00|         34.0|         NO2|         36.0|         30.0|Maharashtra|More Chowk Waluj,...|
|Aurangabad|  India|905|2021-01-14 04:00:00|          3.0|         NH3|          3.0|          3.0|Maharashtra|More Chowk Waluj,...|
|Aurangabad|  India|906|2021-01-14 04:00:00|         19.0|         SO

In [11]:
df.printSchema()

root
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- lst_update: timestamp (nullable = true)
 |-- pollutant_avg: double (nullable = true)
 |-- pollutant_id: string (nullable = true)
 |-- pollutant_max: double (nullable = true)
 |-- pollutant_min: double (nullable = true)
 |-- state: string (nullable = true)
 |-- station: string (nullable = true)



In [12]:
df.write.format("mongo").option("uri","mongodb://localhost:27017/AQIdata.Realtime").mode("append").save()

In [14]:
df.count()

231